In [205]:
import pickle
import pandas as pd
import os
import git
import numpy as np
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
## tokenize the text..
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer as keras_Tokenizer
from sklearn.metrics import classification_report
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tweet_tknzr = TweetTokenizer()
## keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.models import Model, Input 
from keras.models import Sequential
from keras.layers import Dense, concatenate
from keras.layers import Flatten
from keras.layers import SpatialDropout1D
from keras.layers import Embedding 
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Average
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from keras.models import Model, Input
from keras import regularizers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

In [2]:
## Read the labelled files and the poly_user
def get_git_root(path):
	git_repo = git.Repo(path, search_parent_directories=True)
	git_root = git_repo.git.rev_parse("--show-toplevel")
	return git_root

In [7]:
top_dir = os.path.join(get_git_root(os.getcwd()))
model_dir = os.path.join(get_git_root(os.getcwd()),"models")
input_dir = os.path.join(get_git_root(os.getcwd()),"input")
poly_dir = os.path.join(model_dir,"poly_users")
embeddings_dir = os.path.join(input_dir,"embeddings")
poly_users = pickle.load(open(os.path.join(poly_dir,"poly_users.pkl"),"rb"))
regular_users = pickle.load(open(os.path.join(poly_dir,"regular_users.pkl"),"rb"))
mono_users = pickle.load(open(os.path.join(poly_dir,"mono_users.pkl"),"rb"))

In [8]:
## input data :juul_data 
juul_data = pd.read_csv(os.path.join(input_dir,"juul_data.csv"),lineterminator="\n")
juul_data["tweetCreatedAt"] = pd.to_datetime(juul_data["tweetCreatedAt"])
juul_data = juul_data.loc[juul_data.userID.isin(regular_users)]

# ## look at the weed first and juul_first tweet
first_data = pd.read_csv(os.path.join(input_dir,"user_first.csv"),lineterminator="\n")
first_data = first_data.loc[first_data.userID.isin(regular_users)]  # reg users
first_data["weed_first"] = pd.to_datetime(first_data["weed_first"])
first_data["juul_first"] = pd.to_datetime(first_data["juul_first"])

embedding_file = os.path.join(embeddings_dir,"glove.twitter.27B.100d.txt")

In [9]:
## removing the rewteet text column completely
ids_ = juul_data.loc[juul_data['retweetText'].notnull()]["tweetId"]
juul_data.loc[juul_data.tweetId.isin(ids_),"tweetText"] = juul_data.loc[juul_data.tweetId.isin(ids_)]["retweetText"]
juul_data = juul_data.drop(["hashtags","retweetText"],axis=1)
len(juul_data) ## 1353987 (sanity check)

1353987

In [10]:
len(juul_data) ## 1353987
len(juul_data.userID.unique()) ## 673724  reg users

673724

In [ ]:
## @ returns the data in that year
def get_year_data(year):
    print("year",year)
    users_ = list(first_data["userID"].loc[ 
        ((first_data.juul_first.dt.year <= year)  & (first_data.juul_first.dt.year > (year-1)))
        & ((first_data.weed_first.dt.year == (year + 1)) | (pd.isnull(first_data.weed_first))) ## weed data after 2015
                ]) # users who will change after september

    poly_turn = list(first_data["userID"].loc[
        (first_data.juul_first.dt.year <= year)  & 
        ((first_data.weed_first.dt.year == (year +1 )))]) ## for labelling based on the next year


    print("users that will change",len(poly_turn))
    print("total users",len(users_))

    ## getting the input data
    data_ = juul_data.loc[juul_data.userID.isin(users_)]
    print("length of data",len(data_))

    ## get label - they reamain same for this task as the no of users, we only change the tweets data
    users_ = data_.userID.unique()
    users_lbl = pd.DataFrame(users_,columns=["userID"])
    users_lbl["label"] = 0  ## initialize
    users_lbl.loc[users_lbl.userID.isin(poly_turn) , "label"] = 1
    len(users_lbl.loc[users_lbl.label == 1])  ## sanity check
    return ((year,data_,users_lbl))

In [119]:
## HELPER FUCNTIONS
# setup env
import nltk
from numpy import zeros
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tweet_tknzr.tokenize(sentence)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)] ## remove punctuations
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_max_length(df):
    ## max_length
    lengths = df["tweetText"].progress_apply(get_length)
    max_len = int(lengths.quantile(0.95))
    return (max_len)

def get_length(s):
    a = list(s.split())
    return(len(a))

def clean_text(text):
    text = re.sub(r'(https?://\S+)', "", text) ## remove url
    text = re.sub(r'(\@\w+)', "author",text)   ## remove @ mentions with author
    text = re.sub(r'(@)', "",text)             ## remove @ symbols
    text = re.sub(r'(author)',"",text)         ## remove author
    text = re.sub(r'(#)', "",text)             ## removing the hashtags signal 
    text = re.sub(r'(RT )', "",text)         ## remove the retweet info as they dont convey any information
    text = re.sub(r'(^:)',"",text)
    text = text.rstrip() 
    text = text.lstrip()
    return(text)

## returns the emnbedding matrix for the lstm model
def get_embedding_matrix(vocab_size,dimension,embedding_file,keras_tkzr):
    word2vec = get_word2vec(embedding_file)
    from numpy import zeros
    embedding_matrix = zeros((vocab_size, dimension))
    for word, i in keras_tkzr.word_index.items():
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/drew_william2345/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/drew_william2345/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [266]:
from sklearn.model_selection import cross_val_score

def get_baseline_scores(X_train,X_test,Y_train,Y_test):
    
    print("training the models")
    print("svm")
    svm = LinearSVC(C=1,verbose=1)
    svm.fit(X_train,Y_train)
    svm_pred = svm.predict(X_test)
    svm_score = precision_recall_fscore_support(Y_test,svm_pred,average=None)[2]  # return the f-score
    svm_f1 =  cross_val_score(svm, X_test, Y_test, cv=5,scoring='f1_macro').mean()
    print('svm cross val score mean',svm_f1,'\n')
    
    print("random_forest")
    rf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
    rf.fit(X_train,Y_train)
    rf_pred = rf.predict(X_test)
    rf_score = precision_recall_fscore_support(Y_test,rf_pred,average=None)[2]
    rf_f1  = cross_val_score(rf, X_test, Y_test, cv=5,scoring='f1_macro').mean()
    print('rf cross val score mean',rf_f1,'\n')
    
    print("xgBoost")
    xgb = XGBClassifier()
    xgb.fit(X_train, Y_train)
    xgb_pred = xgb.predict(X_test)
    xgb_score = precision_recall_fscore_support(Y_test,xgb_pred,average=None)[2]
    xgb_f1 =  cross_val_score(xgb, X_test, Y_test, cv=5,scoring='f1_macro').mean()
    print('xgb corss val score mean',xgb_f1,'\n')
    
    y_pred = [1 for x in range(len(Y_test))]
    print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')
    maj_score = precision_recall_fscore_support(Y_test,y_pred,average=None)[2]
    
    models = {0:[svm_pred,svm,"svm"], 1:[rf_pred,rf,"rf"] ,2:[xgb_pred,xgb,"xgb"]}
    model_idx = np.argmax([svm_f1,rf_f1,xgb_f1])  ## get the best performing model
    
    print("selecting the best model",models[model_idx][2])

    print("job finished")
    all_scores = {
        'svm': [svm,svm_score],
        'rf' : [rf,rf_score],
        'xg_boost': [xgb,xgb_score],
        'maj': [maj_score],
    }
    return (all_scores,models[model_idx])

def prepare_data(input_data,users_labelled):
    ## preapring the user data
    user_data = input_data.groupby(by="userID").agg({  'tweetText' : 'count',
                                                       'followersCount': 'first',
                                                       'friendsCount' : 'first',
                                                       'statusesCount' : 'first',
                                                       'favourites_count' : 'first',
                                                       'listedCount' : 'first',
                                                      })
    user_data = user_data.rename(columns={'tweetText': 'tweetCount'})
    
    # preapring text
    tweet_data = input_data.groupby(by="userID")["tweetText"].apply(lambda x: "%s" % ' '.join(x)).reset_index()
    ## cleaning the text
    tweet_data["tweetText"] = tweet_data["tweetText"].progress_apply(clean_text)
    tweet_data["tweetText"] = tweet_data["tweetText"].progress_apply(get_tokens).str.join(" ")
    
    ## merging the text and user data
    final_data = user_data.join(tweet_data.set_index("userID"),on="userID",how="inner").reset_index()
    final_data = final_data.fillna(0)   
    
    ## extract the labels
    y = list(final_data.join(users_labelled.set_index("userID"),on="userID",how="inner")["label"]) 
    print("downsampling")
    
    ## downsampling based on userIDS
    userIDs = np.array(list(final_data.userID)).reshape(-1,1)
    rus = RandomUnderSampler(random_state=0)
    rus.fit(userIDs,y)
    userIDs , y_sam = rus.fit_sample(userIDs,y)
    input_data = (final_data.loc[final_data.userID.isin(userIDs.flatten())])
    print("downsampled data length",len(input_data))
    
    print("train-test split")
    train_data,test_data,Y_train,Y_test  = train_test_split(input_data, y_sam,test_size =0.20,random_state= 4,shuffle=True ,
                                                          stratify= y_sam)
    return(train_data,test_data,Y_train,Y_test)

## return user fatures
def prepare_user_features(input_):
    user_data = input_[["followersCount","friendsCount","statusesCount"
                          ,"favourites_count","listedCount","tweetCount",]]
    ## followerss/ friends ration
    user_data["ff_ratio"] = user_data["followersCount"] / user_data["friendsCount"]

    ## using log of each of the columns
    user_data[["followersCount","friendsCount"
          ,"statusesCount","favourites_count",
          "listedCount"]] = np.log(user_data[["followersCount","friendsCount","statusesCount","favourites_count",
                                        "listedCount"]])
    
    user_data["unigrams"] = list(input_["tweetText"].apply(get_length))

    ## replace the na and inf values 
    user_data = user_data.replace([np.inf, -np.inf], np.nan)
    user_data.replace(np.nan,0)
    user_data = user_data.fillna(0)

    ## normalizing the values
    user_data = (user_data - user_data.min())/(user_data.max()-user_data.min())
    user_data = user_data.replace([np.inf, -np.inf], np.nan)
    user_data = user_data.fillna(0)

    X = user_data.values
    return (X,user_data)

def run_user_features(train_data,test_data,Y_train,Y_test):
    X_train,_ = prepare_user_features(train_data)
    X_test,_ = prepare_user_features(test_data)
    
    scores,best_model = get_baseline_scores(X_train,X_test,Y_train,Y_test)
    return(scores,best_model[0],best_model[1])

##models

## @ return a trained svm model
def svm_wrapper(X_train,Y_train):
    svm = LinearSVC(C=1, verbose=1)
    svm.fit(X_train, Y_train)
    return svm

def training_plot(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

def run_lstm(train_data,test_data,Y_train,Y_test,dimension,epoch,metrics,weight=None):
    scores = []
    ## print winodow , max_len for analysis purpose
    max_len = get_max_length(train_data)
    if max_len > 60:
        max_len = 60
    print("max_length",max_len)

    ## prepare the tokenizer
    print("preparing the tokenizer")
    keras_tkzr = keras_Tokenizer()
    keras_tkzr.fit_on_texts(train_data["tweetText"])
    vocab_size = len(keras_tkzr.word_index) + 1
    print("vocalb",vocab_size)

    ## embedding matrix
    print("creating glove embeddign matrix")
    embedding_matrix = get_embedding_matrix(vocab_size,dimension,embedding_file,keras_tkzr) ## tokenizer contains the vocalb info

    ## encoding the docs
    print("encoding the data")
    encoded_docs = keras_tkzr.texts_to_sequences(train_data["tweetText"])
    X_train = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))

    ## encoding the test data
    encoded_docs = keras_tkzr.texts_to_sequences(test_data["tweetText"])
    X_test = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))

    print("X-train",X_train.shape)
    print("X-test",X_test.shape)

    ## getting the user features
    X_train_user,_ = prepare_user_features(train_data)
    X_test_user,_ = prepare_user_features(test_data)
    
    user_feat_len  = (X_train_user.shape[1])
    print("creating lstm model")
    model = create_model(max_len,user_feat_len,vocab_size,dimension,embedding_matrix)

    print("training the model with balance dataset")
    history = model.fit([X_train,X_train_user],Y_train,validation_split=0.25 , nb_epoch = epoch, 
                        verbose = 1,batch_size=32,class_weight= None,)

    ##plotting trainin validation - no point as we dont want ot look at accuarcy
    training_plot(history)


    scores = get_cross_val_score(train_data,Y_train,n_splits=5,nb_epoch=epoch)

    print("generating classfication report")
    loss, accuracy = model.evaluate([X_test,X_test_user], Y_test, verbose=2)
    print('Accuracy: %f' % (accuracy*100))
    ## lstm model
    temp = model.predict([X_test,X_test_user])
    y_pred = [np.argmax(value) for value in temp]  ## sigmoid
    print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

    print("lstm cross val score ",np.array(scores).mean() )

    print("job finished")
    return (scores,y_pred,model,keras_tkzr,max_len)

## @ return a trained svm model on text features for LR
def run_text_features(train_data,test_data,Y_train,Y_test):
    tf_idf = TfidfVectorizer(sublinear_tf=True)
    tf_idf.fit(train_data["tweetText"])  ## fit on train data
    
     ## transform train and test data
    X_test = tf_idf.transform(test_data["tweetText"])
    X_train = tf_idf.transform(train_data["tweetText"])
    
    
    ## reduce the dimesionality
    svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
    svd.fit(X_train)  
    X_train = svd.transform(X_train)
    X_test = svd.transform(X_test)
    
    scores,best_model = get_baseline_scores(X_train,X_test,Y_train,Y_test)
    return (scores,best_model[0],best_model[1],tf_idf,svd)

def cal_text_pred(test_data,Y_test,model,tf_idf,svd):
    X_test = tf_idf.transform(test_data["tweetText"])
    X_test = svd.transform(X_test) ## reduce the dimensionality
    y_pred = model.predict(X_test)
    return y_pred

def cal_lstm_pred(test_data,Y_test,model,keras_tkzr,max_len):
     ## encoding the test data
    encoded_docs = keras_tkzr.texts_to_sequences(test_data["tweetText"])
    X_test = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))
    X_test_user,_ = prepare_user_features(test_data)
    ## calculate the model predictions
    temp = model.predict([X_test,X_test_user])
    y_pred = [np.argmax(value) for value in temp]  ## sigmoid
    
    return y_pred

def cal_user_pred(test_data,Y_test,model):
    X_test,_ = prepare_user_features(test_data)
    y_pred = model.predict(X_test)
    return y_pred

# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(file_path, "r")
    if (file):
        word2vec = dict()
#         split = file.read().splitlines()
        for line in file:
            split_line = line.split(' ')
            key = split_line[0] # the first word is the key
            value = np.array([float(val) for val in split_line[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")
        
from keras.callbacks import Callback
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_predict = np.array([np.argmax(value) for value in val_predict])
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print ("— val_f1: %f — val_precision: %f — val_recall %f"%(_val_f1, _val_precision, _val_recall))
        print('  Classification Report:\n',classification_report(val_targ,val_predict),'\n')
        return

In [253]:
## handle two different inputs and then concatenate them (user and text features)
## input = [words_in,user_in]
def create_model(max_len,user_feature_len,vocalb_size,dimension,embedding_matrix):
    ## handle text features..
    words_in = Input(shape=(max_len,))
    emb_word = Embedding(vocab_size,dimension,weights=[embedding_matrix],input_length=max_len)(words_in) 
    lstm_word =  Bidirectional (LSTM (100,return_sequences=False,dropout=0.50,kernel_regularizer=regularizers.l2(0.01)),merge_mode='concat')(emb_word)
    lstm_word = Dense(user_feature_len,activation='relu')(lstm_word)
    
    ## takes the user features as input
    user_input = Input(shape=(user_feature_len,))
    
    ## concatenate both of the features
    modelR = concatenate([lstm_word, user_input])
    # modelR = SpatialDropout1D(0.1)(modelR)
    output = Dense(2,activation='softmax')(modelR)
    model = Model([words_in,user_input],output)
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model
    

In [254]:
## return cross_val mean score for each 
def get_cross_val_score(train_data,Y_train,n_splits,nb_epoch):
    scores = []
    train_ids = list(train_data.index)
    kFold = StratifiedKFold(n_splits=n_splits)
    for train, test in kFold.split(train_ids,Y_train):
        X_train_user,_ = prepare_user_features(train_data.loc[train])
        X_test_user,_ = prepare_user_features(train_data.loc[test])

        encoded_docs = keras_tkzr.texts_to_sequences(train_data.loc[train]["tweetText"])
        X_train = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))
        encoded_docs = keras_tkzr.texts_to_sequences(train_data.loc[test]["tweetText"])
        X_test = (pad_sequences(encoded_docs, maxlen=max_len, padding='post'))

        history = model.fit([X_train,X_train_user],Y_train[train],validation_split=0.25 , nb_epoch = epoch, 
                        verbose = 1,batch_size=32,class_weight= None,)
        training_plot(history)

        ## prediction
        temp = model.predict([X_test,X_test_user])
        y_pred = [np.argmax(value) for value in temp]  ## sigmoid
        f1 = precision_recall_fscore_support(Y_train[test],y_pred,average=None)[2]
        print(f1)
        print('  Classification Report:\n',classification_report(Y_train[test],y_pred),'\n')
        scores.append(f1)
    score1 = np.mean([ele[0] for ele in scores])
    score2 = np.mean([ele[1] for ele in scores])
    return (score1,score2)

In [259]:
## gathering data in different range of intervals
bucket_data = []
bucket_data.append(get_year_data(2015))
bucket_data.append(get_year_data(2016))
bucket_data.append(get_year_data(2017))

year 2015
users that will change 97
total users 1049
length of data 2944
year 2016
users that will change 313
total users 1636
length of data 6475
year 2017
users that will change 9525
total users 40236
length of data 111735


In [ ]:
## running by year
dimension = 100
for year,data,users_labelled in bucket_data:
    print("************")
    print("preparing the data")
    train_data,test_data,Y_train,Y_test = prepare_data(data,users_labelled)
    train_data = train_data.reset_index(drop=True)
    print("runnning classfier to get user features")
    
    scores,y_pred,lstm_model,keras_tkzr,max_len = run_lstm(train_data,test_data,Y_train,Y_test,
                                                      dimension,epoch,metrics,weight=None)
    
    ## getting predicting on test data
    y_pred = cal_lstm_pred(test_data,Y_test,lstm_model,keras_tkzr,max_len)
    print('  Classification Report test data:\n',classification_report(Y_test,y_pred),'\n')
    
    





  0%|          | 0/1049 [00:00<?, ?it/s]



100%|██████████| 1049/1049 [00:00<00:00, 49708.23it/s]



  0%|          | 0/1049 [00:00<?, ?it/s]

************
preparing the data






 24%|██▍       | 251/1049 [00:00<00:00, 2506.27it/s]



 53%|█████▎    | 557/1049 [00:00<00:00, 2618.00it/s]



 74%|███████▍  | 780/1049 [00:00<00:00, 2487.97it/s]



 96%|█████████▌| 1005/1049 [00:00<00:00, 2411.27it/s]



100%|██████████| 1049/1049 [00:00<00:00, 2429.72it/s]



  0%|          | 0/155 [00:00<?, ?it/s]



100%|██████████| 155/155 [00:00<00:00, 51682.73it/s]

downsampling
downsampled data length 194
train-test split
runnning classfier to get user features
max_length 60
preparing the tokenizer
vocalb 2073
creating glove embeddign matrix
